# Training LemurNet
For explanations on this notebook, check out https://www.lemurnet.org/blog_training/

In [1]:
import tensorflow as tf
import os
from multiprocessing import Pool
import keras

print("Tensorflow version:", tf.__version__)
print("Keras version:", keras.__version__)

import warnings

warnings.filterwarnings("ignore")

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import Sequence
from keras import layers, models, callbacks, optimizers

Using TensorFlow backend.


Tensorflow version: 1.11.0
Keras version: 2.2.4


In [ ]:
# The path to your dataset, must contian subfolders "train" and "validation"
# each of which must contain one folder per class
base_dir = "/path/to/your/dataset"

train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")

classes = os.listdir(train_dir)
classes.sort()
N_CLASSES = len(classes)

In [3]:
from keras.applications import mobilenetv2

preprocess_input = mobilenetv2.preprocess_input

train_datagen = ImageDataGenerator(
     rotation_range=20,
     width_shift_range=0.2,
     height_shift_range=0.2,
     shear_range=0.1,
     zoom_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    preprocessing_function=preprocess_input,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
conv_base = mobilenetv2.MobileNetV2(
    include_top=False, pooling="avg", input_shape=(224, 224, 3), weights="imagenet"
)

9412608/9406464 [==============================] - 1s 0us/step


In [14]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=conv_base.input_shape[1:-1],
    batch_size=17,
    class_mode="categorical",
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=conv_base.input_shape[1:-1],
    batch_size=27,
    class_mode="categorical",
)

Found 7599 images belonging to 35 classes.
Found 2052 images belonging to 35 classes.


In [21]:
conv_base.trainable = False
for layer in conv_base.layers:
    layer.trainable = False

In [22]:
features = conv_base.output
features = layers.Dropout(0.5, name="dropout")(features)
predictions = layers.Dense(N_CLASSES, activation="softmax", name="predictions")(
    features
)

In [23]:
model = models.Model(conv_base.input, predictions)

In [24]:
cb = [
    callbacks.ModelCheckpoint(
        "lemurnet_mobilenet.h5", monitor="val_acc", verbose=0, save_best_only=True
    ),
    callbacks.EarlyStopping(monitor="val_acc", patience=3, verbose=1),
    callbacks.TensorBoard(log_dir="./logs", write_images=True),
    callbacks.ReduceLROnPlateau(monitor="val_acc", factor=0.8, patience=1, verbose=1),
]

In [25]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
# model.summary()

In [26]:
history = model.fit_generator(
    train_generator,
    epochs=50,
    steps_per_epoch=7599 // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=2052 // validation_generator.batch_size,
    verbose=1,
    callbacks=cb,
    max_queue_size=2 * train_generator.batch_size,
    use_multiprocessing = True,
    workers=4
)

model.load_weights("lemurnet_mobilenet.h5")

Epoch 1/50
447/447 [==============================] - 113s 253ms/step - loss: 2.9519 - acc: 0.2523 - val_loss: 2.9603 - val_acc: 0.2515
Epoch 2/50
447/447 [==============================] - 100s 224ms/step - loss: 2.1875 - acc: 0.4118 - val_loss: 2.9665 - val_acc: 0.2641
Epoch 3/50
447/447 [==============================] - 101s 226ms/step - loss: 2.1855 - acc: 0.4256 - val_loss: 2.8611 - val_acc: 0.3367
Epoch 4/50
447/447 [==============================] - 97s 218ms/step - loss: 2.1238 - acc: 0.4380 - val_loss: 2.8970 - val_acc: 0.3221

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.
Epoch 5/50
447/447 [==============================] - 100s 224ms/step - loss: 2.0878 - acc: 0.4526 - val_loss: 3.0802 - val_acc: 0.3294

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0006400000303983689.
Epoch 6/50
447/447 [==============================] - 99s 222ms/step - loss: 1.9888 - acc: 0.4698 - val_loss: 2.6814 - val_acc: 0.3860
Epoch 7/50
447/447 [=====

In [28]:
conv_base.trainable = True
set_trainable = False

# make block_16_expand and subsequent layers trainable for fine-tuning
for layer in conv_base.layers:
    if layer.name == "block_15_expand":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False


In [29]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(lr=1e-5),
    metrics=["acc"],
)

In [30]:
cb = [
    callbacks.ModelCheckpoint(
        "lemurnet_mobilenet_finetuning.h5",
        monitor="val_acc",
        verbose=0,
        save_best_only=True,
    ),
    callbacks.EarlyStopping(monitor="val_acc", patience=5, verbose=1),
    callbacks.TensorBoard(log_dir="./logs_finetuning", write_images=True),
    callbacks.ReduceLROnPlateau(monitor="val_acc", factor=0.5, patience=1, verbose=1),
]

In [32]:
history = model.fit_generator(
    train_generator,
    epochs=50,
    steps_per_epoch=7599 // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=2052 // validation_generator.batch_size,
    verbose=1,
    callbacks=cb,
    max_queue_size=2 * train_generator.batch_size,
    use_multiprocessing = True,
    workers=4
)

model.load_weights("lemurnet_mobilenet_finetuning.h5")

Epoch 1/50
447/447 [==============================] - 115s 257ms/step - loss: 1.7683 - acc: 0.5119 - val_loss: 2.1330 - val_acc: 0.4620
Epoch 2/50
447/447 [==============================] - 101s 227ms/step - loss: 1.5590 - acc: 0.5474 - val_loss: 2.1703 - val_acc: 0.4488

Epoch 00002: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Epoch 3/50
447/447 [==============================] - 103s 230ms/step - loss: 1.5529 - acc: 0.5553 - val_loss: 2.1528 - val_acc: 0.4644
Epoch 4/50
447/447 [==============================] - 99s 221ms/step - loss: 1.4711 - acc: 0.5772 - val_loss: 2.1557 - val_acc: 0.4479

Epoch 00004: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
Epoch 5/50
447/447 [==============================] - 102s 228ms/step - loss: 1.4255 - acc: 0.5922 - val_loss: 1.9970 - val_acc: 0.4776
Epoch 6/50
447/447 [==============================] - 101s 225ms/step - loss: 1.4572 - acc: 0.5739 - val_loss: 2.1465 - val_acc: 0.4825
Epoch 7/50
447/447 [===

model.save("lemurnet.h5")